In [ ]:
class summarizer_TTS:

  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  !pip install stanza torch torchaudio librosa soundfile tqdm pyyaml fast-bss-eval filelock flask fonttools frozenlist espnet pandas indic_unified_parser
  def __init__(self, text, target_language = 'Telugu'):
    self.tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-1.3B") #3.3B is not real time and very slow and 600M is not accurate enough
    self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-1.3B")

    !sudo apt-get install git-lfs
    !git lfs install
    !git clone https://github.com/smtiitm/Fastspeech2_HS.git
    !cd Fastspeech2_HS
    !git lfs fetch --all
    !git lfs pull

  def convert_indic_lang_to_english(self, indic_text, target_language_code = 'eng_Latn'):
    inputs = tokenizer(indic_text, return_tensors="pt")
    translated_tokens = self.model.generate(
        **inputs,
        forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(target_language_code))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text

  def convert_english_to_indic_lang(self, english_text, target_language_code = 'tel_Telu'):
      inputs = tokenizer(english_text, return_tensors="pt")
      translated_tokens = self.model.generate(
          **inputs,
          forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(target_language_code))
      translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
      return translated_text

  def convert(self, text, target_language):






In [3]:
import pandas as pd
import json

with open("database.json", "r", encoding="utf-8") as f:
    data = json.load(f)      # Telugu text loads correctly

df = pd.DataFrame(data)
df.to_csv("user_study.csv",index=False, encoding="utf-8")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-1.3B") #3.3B is not real time and very slow and 600M is not accurate enough
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-1.3B")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def convert_indic_lang_to_english(indic_text, target_language_code = 'eng_Latn'):
    inputs = tokenizer(indic_text, return_tensors="pt")
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_language_code))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text
def convert_english_to_indic_lang(english_text, target_language_code = 'tel_Telu'):
    inputs = tokenizer(english_text, return_tensors="pt")
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_language_code))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text

In [ ]:
!pip install stanza torch torchaudio librosa soundfile tqdm pyyaml fast-bss-eval filelock flask fonttools frozenlist espnet pandas indic_unified_parser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 39.3 MB/s eta 0:00:00


In [ ]:
import re
# If using a POS pipeline, set nlp to your Telugu tagger; else set nlp=None
nlp = None  # replace with stanza or your POS pipeline if available

TOOL_HINTS = ["పనిముట్టు", "సాధనం", "యాప్", "సాఫ్ట్‌వేర్"]
TELUGU = r"[\u0C00-\u0C7F]+"

def highlight_nouns_with_fallback(text):
    spans = []

    # 1) POS-driven spans (optional)
    if nlp is not None:
        doc = nlp(text)
        for s in doc.sentences:
            for w in s.words:
                if w.upos in {"NOUN", "PROPN"} and w.start_char is not None and w.end_char is not None:
                    pre = text[w.start_char-1] if w.start_char > 0 else ""
                    post = text[w.end_char] if w.end_char < len(text) else ""
                    if pre in "'‘’" or post in "'‘’":
                        continue
                    spans.append((w.start_char, w.end_char))

    # 2) Fallback: quote first Telugu token after the earliest tool hint
    hint_pos = min((text.find(h), h) for h in TOOL_HINTS if text.find(h) != -1)[0] if any(text.find(h)!=-1 for h in TOOL_HINTS) else -1
    if hint_pos != -1:
        right = text[hint_pos:]
        m = re.search(TELUGU, right)
        if m:
            s = hint_pos + m.start()
            e = hint_pos + m.end()
            pre = text[s-1] if s > 0 else ""
            post = text[e] if e < len(text) else ""
            if not (pre in "'‘’" or post in "'‘’"):
                spans.append((s, e))

    # Merge overlaps
    spans = sorted(set(spans))
    merged = []
    for s, e in spans:
        if not merged or s > merged[-1][1]:
            merged.append([s, e])
        else:
            merged[-1][1] = max(merged[-1][1], e)

    # Rebuild with quotes
    out, i = [], 0
    for s, e in merged:
        out.append(text[i:s]); out.append("'"); out.append(text[s:e]); out.append("'"); i = e
    out.append(text[i:])
    return "".join(out)

src = "తేలికగా తెలుగులో వ్రాయడానికి ఉపయోగపడే పనిముట్టు లేఖిని. ఇకనుండి మీ బంధుమిత్రులకు తెలుగులో సందేశాలు పంపించుకోవచ్చు! ఈ సేవ ఉచితం!"
print(highlight_nouns_with_fallback(src))


తేలికగా తెలుగులో వ్రాయడానికి ఉపయోగపడే 'పనిముట్టు' లేఖిని. ఇకనుండి మీ బంధుమిత్రులకు తెలుగులో సందేశాలు పంపించుకోవచ్చు! ఈ సేవ ఉచితం!


In [ ]:
s = "తేలికగా తెలుగులో వ్రాయడానికి ఉపయోగపడే పనిముట్టు లేఖిని. ఇకనుండి మీ బంధుమిత్రులకు తెలుగులో సందేశాలు పంపించుకోవచ్చు! ఈ సేవ ఉచితం!"
out = s.replace("లేఖిని", "___TOOL_NAME___", 1)
eng_converted_text = convert_indic_lang_to_english(indic_text = out)
final = eng_converted_text.replace("___TOOL_NAME___", "'లేఖిని'")
print(final)

The tool used to write in Telugu is 'లేఖిని'. From now on, you can send messages to your friends in Telugu! This service is free!


In [ ]:
back_to_indic = convert_english_to_indic_lang(final)
print(back_to_indic)

తెలుగులో రాయడానికి ఉపయోగించే సాధనం లేఖిని. ఇక నుంచి తెలుగులో మీ స్నేహితులకు సందేశాలు పంపవచ్చు! ఈ సేవ ఉచితం!


TTS synthesis using Bhashini-IIT Madras Model

In [ ]:
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/smtiitm/Fastspeech2_HS.git
!cd Fastspeech2_HS
!git lfs fetch --all
!git lfs pull

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Git LFS initialized.
Cloning into 'Fastspeech2_HS'...
remote: Enumerating objects: 4531, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 4531 (delta 249), reused 227 (delta 227), pack-reused 4272 (from 2)
Receiving objects: 100% (4531/4531), 2.94 GiB | 27.02 MiB/s, done.
Resolving deltas: 100% (856/856), done.
Updating files: 100% (519/519), done.
Error downloading object: assamese/female/model/model.pth (ff9efd8): Smudge error: Error downloading assamese/female/model/model.pth (ff9efd82aacade9fcd3fc17e350147d3a272356b1f419872ac9fa525b991b578): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/

Model.pth files are not present in the current git-lfs pull due to free colab disk storage restrictions. We should put them in this downloaded in lang/gender/model/ . I did for telugu/male.

In [ ]:
!cd /content/Fastspeech2_HS && python inference.py \
  --sample_text "తేలికగా తెలుగులో వ్రాయడానికి ఉపయోగపడే పనిముట్టు లేఖిని. ఇకనుండి మీ బంధుమిత్రులకు తెలుగులో సందేశాలు పంపించుకోవచ్చు! ఈ సేవ ఉచితం!" \
  --language telugu \
  --gender male \
  --alpha 1.0 \
  --output_file telugu_male_lekhini.wav

print("Done.")

Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'

Phone dictionary loaded for the following languages: ['odia', 'hindi', 'marathi', 'manipuri', 'punjabi', 'malayalam', 'telugu', 'tamil', 'rajasthani', 'assamese', 'english', 'kannada', 'bengali', 'bodo', 'gujarati']
Loading G2P model... Done!

Phone dictionary loaded for the following languages: ['odia', 'hindi', 'marathi', 'manipuri', 'punjabi', 'malayalam', 'telugu', 'tamil', 'rajasthani', 'assamese', 'english', 'kannada', 'bengali', 'bodo', 'gujarati']
Loading G2P model... Done!

Phone dictionary loaded for the following languages: ['odia', 'hindi', 'marathi', 'manipuri', 'punjabi', 'malayalam', 'telugu', 'tamil', 'rajasthani', 'assamese', 'english', 'kannada', 'bengali', 'bodo', 'gujarati']
Loading G2P model... Done!

Phone dictionary loaded for the following languages: ['odia', 'hindi', 'marathi', 'manipuri', 'punjabi', 'malayalam', 'telugu', 'tamil', 'rajasthani', 'assamese', 'english', 'kannada